# Notebook para generar el dataset

Utilizando la biblioteca Spotipy vamos a generar un csv, haciendo una búsqueda general por año para obtener todas las tracks que fueron publicadas. Para realizar las consultas tomamos el mercado Argentina (AR).
Luego, de cada una de las tracks obtenidas se vovlerá a realizar consultas a la Web API de Spotify para obtener información un poco más técnica de cada una de las tracks.
Con todos esos datos, se generará un DataSet y se lo almacenará en un archivo CSV para su uso posterior.

In [1]:
import spotipy #pip install spotipy --upgrade
from spotipy.oauth2 import SpotifyClientCredentials#para gestionar la autenticacion contra la API de Spotify
import pandas as pd
import numpy as np

# Inicializamos Spotipy

##### Credenciales para conectarse a la web api de Spotify

In [2]:
SPOTIPY_CLIENT_ID = 'e8b2379bdd054eb993035179ae40e066'
SPOTIPY_CLIENT_SECRET = 'f191a3a661cc47d68b92a296669dde05'#este valor puede ser cambiado desde la app

##### Incializamos el objeto sp para poder realizar las requests

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)

# Realizamos las consultas por año

##### Seteamos los años en que las tracks fueron publicadas

In [4]:
years = list(range(2017, 2021))#excluye el final

##### Seteamos la cantidad de máxima de tracks que queremos tomar como muestra para cada uno de los años

In [5]:
max_cantidad_tracks_anio = 2000

##### Seteamos un offset inicial para comenzar a tomar desde ahí los tracks

In [6]:
initial_offset = 0

##### Realizamos la consulta

In [7]:
#declaramos un array de diccionarios donde vamos a ir poniendo los resultados, para luego generar el data frame
data = []

#loopeamos por los años seteados
for year in years:
    print('Año: ' + str(year))    
    
    #configuramos la consulta para el año actual
    search_query = 'year:' + str(year)
    
    #incializamos un contador de tracks, para no traer mas de las que configuramos en max_cantidad_tracks_anio
    tracks_obtenidas = 0        
    
    #definimos una cantidad maxima de reintentos ante errores
    max_retry = 10
    
    #la web api solo permite obtener hasta 50 items a la vez, es por eso que para llegar a cantidad max de tracks
    #por año, se deberá realizar más de una consulta para el mismo año
    while tracks_obtenidas < max_cantidad_tracks_anio:                
        
        #si ya no quedan reintentos, se abandona el año
        if(max_retry<=0):
            print('\nSe consumieron todos los reintentos, se abandona el año.')
            break
        
        try:
            #cuando se hace una consulta a la web api, spotify encuentra por ejemplo 120 resultados
            #pero solo permite traer de a 50 (limit) a la vez, para poder traer todos hay que ir moviéndose en ese
            #resultado con el offset. Para traer las primeras 50, el offset = 0, para traer las siguientes 50, el offset = 50
            result = sp.search(search_query, limit=50, offset=initial_offset+tracks_obtenidas, type='track', market='AR')

            if(result is not None and result['tracks'] is not None and len(result['tracks']['items']) > 0):
                #nos quedamos con las tracks
                tracks_results = list(result['tracks']['items'])

                #por cada uno de los tracks obtenidos, consultamos los datos tecnicos
                #y luego generamos una entrada en el array de diccionarios
                for track_result in tracks_results:
                    track_spotify_id = track_result['id']
                    album = track_result['album']
                    artist = track_result['artists'][0]

                    #nos aseguramos de no agregar mas tracks que las que se establecieron en max_cantidad_tracks_anio
                    if(tracks_obtenidas < max_cantidad_tracks_anio):
                        #buscamos en el array de diccionarios 'data' el artista para saber si ya lo tenemos en memoria o
                        #tendremos que hacer una consulta a la web api para obtener su información
                        datos_artista = [x for x in data if x['artista_id']==artist['id']]
                        
                        if(datos_artista is None or datos_artista == []):
                            #si no se tiene aun el artista se lo va a buscar a spotify
                            artista_web_api_result = sp.artist(artist['id'])
                            datos_artista = {'artista_id': artista_web_api_result['id'], 
                                             'artista_generos': ','.join(artista_web_api_result['genres']),
                                             'artista_followers': artista_web_api_result['followers']['total'] if artista_web_api_result['followers'] is not None and artista_web_api_result['followers'] != [] else -1,
                                             'artista_name': artista_web_api_result['name']}
                        else:
                            #hasta este punto 'datos_artista' es un array de diccionarios, tomamos el primer elemento
                            datos_artista = datos_artista[0]


                        #consultamos a la web api de spotify los datos tecnicos de la track actual                    
                        track_features_result = sp.audio_features(track_spotify_id)

                        if(track_features_result is not None and len(track_features_result) > 0):
                            track_features = track_features_result[0]                    

                            #agregamos los datos al array de diccionarios
                            data.append({'popularity': track_result['popularity'], 'id':track_spotify_id, 
                                         'name':track_result['name'], 'album': album['name'], 'duration_ms':track_result['duration_ms'],
                                         'album_release_date': album['release_date'], 'explicit': track_result['explicit'],
                                         'danceability': track_features['danceability'], 'energy': track_features['energy'],
                                         'key': track_features['key'], 'loudness': track_features['loudness'],
                                         'mode': track_features['mode'], 'speechiness': track_features['speechiness'],
                                         'acousticness': track_features['acousticness'], 'instrumentalness': track_features['instrumentalness'],
                                         'liveness': track_features['liveness'], 'valence': track_features['valence'],
                                         'tempo': track_features['tempo'], 'artista_id': datos_artista['artista_id'],
                                         'artista_generos': datos_artista['artista_generos'], 'artista_followers': datos_artista['artista_followers'],
                                         'artista_name': datos_artista['artista_name'], 'time_signature': track_features['time_signature']
                                        })

                            tracks_obtenidas += 1
                            print(str(tracks_obtenidas) + ' tracks obtenidas!', end='\r')

            else:
                print('Datos vacíos, reintentando')
                #ante cualquier error, bajamos la cantidad de reintentos y seguimos
                max_retry -= 1                    
        
        except Exception as e: 
            print('Error: '+ str(e))
            print('Reintentando')
            #ante cualquier error, bajamos la cantidad de reintentos y seguimos
            max_retry -=1
    print('\nAño completado')
            

Año: 2017
2000 tracks obtenidas!
Año completado
Año: 2018
2000 tracks obtenidas!
Año completado
Año: 2019
2000 tracks obtenidas!
Año completado
Año: 2020
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando
Error: 'NoneType' object is not subscriptable
Reintentando

Se consumieron todos los reintentos, se abandona el año.

Año completado


# Generamos el DataFrame

In [8]:
df_tracks = pd.DataFrame(data)

In [9]:
df_tracks.shape

(6123, 23)

In [10]:
df_tracks.head()

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
0,87,0pqnGHJpmpxLKifKRmU6WP,Believer,Evolve,204346,2017-06-23,False,0.776,0.780,10,...,0.0622,0.0,0.0810,0.666,124.949,53XhwfbYqKCa1cC15pYq2q,"modern rock,rock",27642174,Imagine Dragons,4
1,86,0tgVpDi06FyKpA1z0VMD4v,Perfect,÷ (Deluxe),263400,2017-03-03,False,0.599,0.448,8,...,0.1630,0.0,0.1060,0.168,95.050,6eUKZXaKkcviH0Ku9w2n3V,"pop,uk pop",65537298,Ed Sheeran,3
2,73,2kP6fOqdSQ5CYqVH5z5844,Te Encontre,Versatilidad & Vivencias (Deluxe Edition),209960,2017-11-17,False,0.733,0.670,7,...,0.5250,0.0,0.2330,0.831,119.955,6HWumjoLO58NYInUHk9565,,44866,El Vega,4
3,81,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,340706,2017-05-12,False,0.516,0.595,5,...,0.0275,0.0,0.1090,0.222,119.972,6KImCVD70vtIoJWnq6nGn3,"pop,post-teen pop",7768887,Harry Styles,4
4,85,7qiZfU4dY1lWllzX7mPBI3,Shape of You,÷ (Deluxe),233712,2017-03-03,False,0.825,0.652,1,...,0.5810,0.0,0.0931,0.931,95.977,6eUKZXaKkcviH0Ku9w2n3V,"pop,uk pop",65537298,Ed Sheeran,4


In [11]:
df_tracks.tail(10)

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
6113,10,5cqmMKv08KG8aXBbxr2JjR,Sorry,Radio music 2020,255333,2020-06-26,False,0.637,0.764,8,...,0.0528,0.000000,0.258,0.474,104.277,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6114,3,1JAOqUOGTF24k7dA3ZvwtB,Pray - Acoustic Version,Yoga Playlist 2020,211985,2020-06-26,False,0.592,0.887,11,...,0.0906,0.000005,0.115,0.700,82.000,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6115,3,33WcW1V3x2z0Q0WwR1Pb2I,Fall,Relaxing Sleep Music,248386,2020-06-26,False,0.607,0.754,11,...,0.2590,0.000000,0.111,0.606,82.995,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6116,6,3ErAVAxeasUpESf3UM0ltz,Be Alright,Relaxing Sleep Music,189653,2020-06-26,False,0.803,0.356,5,...,0.5890,0.000000,0.160,0.739,106.950,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6117,4,3XpZXgxkiKxjagglPHeT5W,Be Alright,Yoga Songs 2020,189653,2020-06-26,False,0.803,0.356,5,...,0.5890,0.000000,0.160,0.739,106.950,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6118,6,5IYbFI92iyKBZdBtvH4nNK,Life Is Worth Living,Study Music 2020,234786,2020-06-26,False,0.558,0.296,10,...,0.9070,0.000000,0.108,0.463,76.040,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6119,5,61GxHzG9Q3HhlFLvT1fHlB,Fall,Sunrise Music 2020,248386,2020-06-26,False,0.607,0.754,11,...,0.2590,0.000000,0.111,0.606,82.995,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6120,0,2hOLwtPxWSjKVtNsmyfQV7,Purpose,Study Music 2020,210173,2020-06-26,False,0.481,0.285,0,...,0.9000,0.000000,0.123,0.316,130.021,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6121,1,5hQkFP8ISK51F38SzGoTY4,Purpose,Yoga Playlist 2020,210173,2020-06-26,False,0.481,0.285,0,...,0.9000,0.000000,0.123,0.316,130.021,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4
6122,2,5yRmBdH0FivN76jYyhDGwe,Be Alright,Yoga Playlist 2020,189653,2020-06-26,False,0.803,0.356,5,...,0.5890,0.000000,0.160,0.739,106.950,1uNFoZAHBGtllmzznpCI3s,"canadian pop,pop,post-teen pop",35707716,Justin Bieber,4


# Generamos el CSV

In [12]:
tracks_csv_name = 'tracks.csv'

In [13]:
csv_exists = True

In [14]:
#verificamos si el csv existe, si existe lo abrimos para hacer append del dataframe
#si no existe, se crea el archivo
try:
    df_aux = pd.read_csv(tracks_csv_name, header=0)
except:
    csv_exists = False

In [15]:
if(csv_exists):
    df_tracks.to_csv(tracks_csv_name, mode='a', header=False,index=False,encoding='utf-8')
else:
    df_tracks.to_csv(tracks_csv_name,index=False,encoding='utf-8')